<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using Apache Spark Structured Streaming with SingleStore Notebooks</h1>
    </div>
</div>

In [4]:
!pip cache purge --quiet

In [5]:
!conda install -y --quiet -c conda-forge openjdk=8

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk=8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.12.14 |       hbcca054_0         153 KB  conda-forge
    certifi-2024.12.14         |     pyhd8ed1ab_0         158 KB  conda-forge
    libgcc-14.2.0              |       h77fa898_1         829 KB  conda-forge
    libgcc-ng-14.2.0           |       h69a702a_1          53 KB  conda-forge
    libgomp-14.2.0             |       h77fa898_1         450 KB  conda-forge
    openjdk-8.0.412            |       hd590300_1        88.3 MB  conda-forge
    openssl-3.4.0              |       hb9d3cd8_0         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        92.7

In [6]:
!pip install openai==0.28 --quiet
!pip install nltk --quiet
!pip install pyspark --quiet

In [7]:
import nltk
import openai
import os
import random
import requests
import shutil
import time

from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, udf
from pyspark.sql.types import StringType
from singlestoredb.management import get_secret

In [8]:
os.environ["OPENAI_API_KEY"] = get_secret("OPENAI_API_KEY")

In [9]:
os.makedirs("jars", exist_ok = True)
os.makedirs("data", exist_ok = True)

In [10]:
def download_jar(url, destination):
    response = requests.get(url)
    with open(destination, "wb") as f:
        f.write(response.content)

jar_urls = [
    ("https://repo1.maven.org/maven2/com/singlestore/singlestore-jdbc-client/1.2.4/singlestore-jdbc-client-1.2.4.jar", "jars/singlestore-jdbc-client-1.2.4.jar"),
    ("https://repo1.maven.org/maven2/com/singlestore/singlestore-spark-connector_2.12/4.1.8-spark-3.5.0/singlestore-spark-connector_2.12-4.1.8-spark-3.5.0.jar", "jars/singlestore-spark-connector_2.12-4.1.8-spark-3.5.0.jar"),
    ("https://repo1.maven.org/maven2/org/apache/commons/commons-dbcp2/2.12.0/commons-dbcp2-2.12.0.jar", "jars/commons-dbcp2-2.12.0.jar"),
    ("https://repo1.maven.org/maven2/org/apache/commons/commons-pool2/2.12.0/commons-pool2-2.12.0.jar", "jars/commons-pool2-2.12.0.jar"),
    ("https://repo1.maven.org/maven2/io/spray/spray-json_3/1.3.6/spray-json_3-1.3.6.jar", "jars/spray-json_3-1.3.6.jar")
]

for url, destination in jar_urls:
    download_jar(url, destination)

print("JAR files downloaded successfully")

JAR files downloaded successfully


In [11]:
# Create a Spark session
spark = (SparkSession
             .builder
             .config("spark.jars", ",".join([destination for _, destination in jar_urls]))
             .appName("Spark Streaming Test")
             .getOrCreate()
        )

spark.sparkContext.setLogLevel("ERROR")

24/12/21 03:50:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [12]:
# Download NLTK
nltk.download("punkt_tab")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")
nltk.download("omw")

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data] Downloading package omw to /home/jovyan/nltk_data...


True

In [14]:
# Define the directory to save the files
output_dir = "data"

# Generate meaningful sentences
def generate_meaningful_sentence():
    # Choose a random set of synonyms from WordNet
    synset = random.choice(list(wn.all_synsets()))

    # Generate a sentence
    definition = synset.definition()
    tokens = word_tokenize(definition)

    # Capitalise the first word and end with a period
    tokens[0] = tokens[0].capitalize()
    tokens[-1] = tokens[-1] + "."

    return " ".join(tokens)

# Number of files to generate
num_files = 5

# Number of sentences in each file
num_sentences_per_file = 1

# Generate text files
for i in range(num_files):
    file_path = os.path.join(output_dir, f"file_{i+1}.txt")
    with open(file_path, "w") as file:
        for _ in range(num_sentences_per_file):
            # Generate a meaningful sentence
            sentence = generate_meaningful_sentence()
            file.write(sentence + "\n")

In [15]:
# Specify the directory containing the .txt files
directory = Path("data")

# Loop through each .txt file in the directory
for file_path in directory.glob("*.txt"):
    print(f"File: {file_path}")

    # Read and print the contents of the file
    with file_path.open() as file:
        print(file.read())

    print("----------------------")

File: data/file_1.txt
Engrave or cut by chipping away at a surface.

----------------------
File: data/file_2.txt
Increasing in quantity or value.

----------------------
File: data/file_3.txt
Any agent that reduces milk secretion ( as given to a woman who is not breast feeding ).

----------------------
File: data/file_4.txt
A form of benzodiazepine ( trade name Halcion ) frequently prescribed as a sleeping pill ; usually given to people who have trouble falling asleep.

----------------------
File: data/file_5.txt
Give off unsteady sounds , alternating in amplitude or frequency.

----------------------


In [16]:
%%sql
DROP DATABASE IF EXISTS spark_demo;
CREATE DATABASE IF NOT EXISTS spark_demo;

1 rows affected.

++
||
++
++

In [17]:
%%sql
USE spark_demo;

DROP TABLE IF EXISTS streaming;
CREATE TABLE IF NOT EXISTS streaming (
     value TEXT,
     file_name TEXT,
     embedding VECTOR(1536) NOT NULL
);

++
||
++
++

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [19]:
from sqlalchemy import *

db_connection = create_engine(connection_url)
url = db_connection.url

In [20]:
password = get_secret("password")
host = url.host
port = url.port
cluster = host + ":" + str(port)

In [21]:
spark.conf.set("spark.datasource.singlestore.ddlEndpoint", cluster)
spark.conf.set("spark.datasource.singlestore.user", "admin")
spark.conf.set("spark.datasource.singlestore.password", password)
spark.conf.set("spark.datasource.singlestore.disablePushdown", "false")

In [22]:
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Generate embeddings for text
def generate_embeddings(text):
    # Generate embeddings for text using OpenAI
    return openai.Embedding.create(
        input = text,
        engine = "text-embedding-3-small"
    ).data[0].embedding

# Register the function as a UDF
generate_embeddings_udf = udf(generate_embeddings, StringType())

In [23]:
input_dir = output_dir

# Read from the directory
df = (spark.readStream
    .format("text")
    .option("path", input_dir)
    .load()
    .withColumn("file_name", input_file_name())
)

# Apply the function to the DataFrame to generate embeddings for each row
df_with_embeddings = df.withColumn("embedding", generate_embeddings_udf("value"))

# Write each batch of data to SingleStore
def write_to_singlestore(df_with_embeddings, epoch_id):
    (df_with_embeddings.write
         .format("singlestore")
         .option("loadDataCompression", "LZ4")
         .mode("append")
         .save("spark_demo.streaming")
    )

# Write the streaming DataFrame to SingleStore using foreachBatch
query = (df_with_embeddings.writeStream
    .foreachBatch(write_to_singlestore)
    .start()
)

# Wait for the query to finish processing
while query.isActive:
    time.sleep(1)
    if not query.status["isDataAvailable"]:
        query.stop()

In [24]:
%%sql
USE spark_demo;

SELECT
    SUBSTR(value, 1, 30) AS value,
    SUBSTR(file_name, LENGTH(file_name) - 9) AS file_name,
    SUBSTR(JSON_ARRAY_UNPACK(embedding :> BLOB), 1, 50) AS embedding
FROM streaming;

5 rows affected.

value,file_name,embedding
A form of benzodiazepine ( tra,file_4.txt,"[0.00202737423,0.0188153591,-0.0181082636,0.064787"
Increasing in quantity or valu,file_2.txt,"[-0.00131295784,-0.019618636,0.0321576819,0.041007"
"Give off unsteady sounds , alt",file_5.txt,"[-0.000452552777,0.00247969362,-0.00644094311,0.01"
Any agent that reduces milk se,file_3.txt,"[-0.000569961907,-0.0157558117,-0.0436274782,0.031"
Engrave or cut by chipping awa,file_1.txt,"[0.0102624856,0.0265935604,-0.00223223562,0.003282"


In [25]:
spark.stop()

## Cleanup

In [27]:
shutil.rmtree("jars")
shutil.rmtree("data")
shutil.rmtree("nltk_data")

In [28]:
%%sql
DROP TABLE IF EXISTS streaming;

++
||
++
++

In [29]:
%%sql
DROP DATABASE IF EXISTS spark_demo;

++
||
++
++